### Read 'profiles' and 'companies' datasets 

In [0]:
import pyspark
from pyspark.sql import SparkSession

# Creating Spark Session
spark = SparkSession.builder.getOrCreate()

profiles = spark.read.parquet('/linkedin/people')
companies = spark.read.parquet('/linkedin/companies')

### Read additional data

In [0]:
# summary_skills as spark
job_summary_skills_spk = spark.read.format("csv").option("header", "true") \
    .option("delimiter", ",") \
    .option("multiline", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("encoding", "UTF-8")\
    .load("dbfs:/FileStore/shared_uploads/adangaben@campus.technion.ac.il/job_summary_skills_1.csv")

# count for 'job_summary_skills_spk': 440769

In [0]:
# sample 100,000 rows from the 'job_summary_skills_spk' dataframe
job_summary_skills_spk = job_summary_skills_spk.sample(False, 100040/440769, seed=85)

In [0]:
import pandas as pd

# summary_skills as pandas
job_summary_skills_pd = pd.read_csv
("/dbfs/FileStore/shared_uploads/adangaben@campus.technion.ac.il/job_summary_skills_1.csv")

# shape for 'job_summary_skills_pd': (440769, 3)

In [0]:
# display(job_summary_skills_pd)

### Attractivity Score Analysis

In [0]:
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, count, lower, regexp_replace, size, 
round, sum, isnan, explode, lit, udf, struct, row_number, length, split, avg, max, first
import re

##### Measure 1: summary length

In [0]:
# count summary length
job_summary_skills_spk = job_summary_skills_spk.withColumn("summary_length", size(split("job_summary", " ")))

In [0]:
# display(job_summary_skills_spk.select('job_summary', 'summary_length').limit(100))

##### Measure 2: new line count

In [0]:
# count newlines
job_summary_skills_spk = job_summary_skills_spk.withColumn
("summary_newline_count", size(split(col("job_summary"), r"\n")) - 1)

In [0]:
# display(job_summary_skills_spk.select('job_summary', 'summary_newline_count').limit(100))

##### Measure 3: sentiment analysis

In [0]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import sparknlp

spark = sparknlp.start()

In [0]:
# FIRST ATTEMPT: VADER sentiment intensity analyzer

# NOTE: if this error appears: PicklingError: args[0] from __newobj__ args has the wrong class
# just run the cell again

# define a UDF for text preprocessing
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    return lemmatized_tokens

preprocess_udf = udf(preprocess_text, StringType())

# apply text preprocessing to the 'job_summary' column
job_summary_skills_spk = job_summary_skills_spk.withColumn
("preprocessed_summary", preprocess_udf(col("job_summary")))

# initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# define a UDF for sentiment analysis
def analyze_sentiment(tokens):
    # join the preprocessed tokens back into a string
    text = " ".join(tokens)
    # perform sentiment analysis using VADER
    sentiment_scores = sid.polarity_scores(text)
    # return the compound sentiment score
    return sentiment_scores['compound'] 

sentiment_udf = udf(analyze_sentiment, StringType())

# apply sentiment analysis to the preprocessed text
job_summary_skills_spk = job_summary_skills_spk.withColumn
("sentiment_score", sentiment_udf(col("preprocessed_summary")))

# drop irrelevant columns
job_summary_skills_spk = job_summary_skills_spk.drop('preprocessed_summary')

In [0]:
# FIRST ATTEMPT RESULTS
display(job_summary_skills_spk.select('job_summary', 'sentiment_score', 'summary_length').limit(10))

job_summary sentiment_score summary_length Want to learn more about this role and Jobot? Click our Jobot logo and follow our LinkedIn page!
Job details
Lead Superintendent Role with a National well regarded Real Estate Developer, all LOCAL Multi-Family Projects, Great Benefits, Bonus and Truck Allowance!
This Jobot Job is hosted by Ben Kimbrel
Are you a fit? Easy Apply now by clicking the "Easy Apply" button and sending us your resume.
Salary $120,000 - $150,000 per year
A Bit About Us
We are a National Multi-Family Real Estate Developer with a large presence in Florida. Our core values are centered around our people and our clients. We believe that putting your employees first is not only the right thing to do, but it also gives us the competitive edge to ensure our clients have an excellent experience! Over the years our reputation for providing our people with the very best has caught the eye of some reputable companies who have chosen to use us as their Prime Contractor! We build a variety of Multi-Family projects including wood-frame over podium and concrete multi story high-rises.
If you are a Construction Superintendent with 5+ years of multi-family experience then please apply!
Why join us?
Do you want to work on great projects and be part of a top Firm!?
Competitive Base Salary!
Competitive Bonus Package!
Extremely Competitive Benefits Package!
Flexible Work Schedules!
Company Truck/Truck Allowance!
Local Work to the Fort Myers Area!
Job Details
Is your background a fit? Apply if you meet this criteria
5+ years’ experience in Construction field supervision
Experience w/ multi-family ground up construction
Experience with projects $20M+
We can offer you the opportunity to work on great Projects and make a meaningful impact on today's society and the next generation! Help us revolutionize the construction industry and enjoy accelerated career growth with a great company!
Interested in hearing more? Easy Apply now by clicking the "Easy Apply" button.
Want to learn more about this role and Jobot?
Click our Jobot logo and follow our LinkedIn page!
Show more
Show less 0.9979 312 Job Details
Description
Join Our Award Winning Team!
With flexible schedules, team member discounts, scholarship opportunities, and fun recognition programs, Delta Sonic offers a variety of career paths for you to grow with us. Recognized nationally as a Top Place to Work, we take great pride in providing outstanding opportunities and a positive work culture.
Team Members Enjoy
Career opportunities
Flexible schedules
Team member discounts
Scholarship opportunities
Benefit options
A fun, team-oriented culture
Equal opportunity employer
Responsibilities
As a front-line representative of the Delta Sonic Team, Customer Advisors are responsible for greeting all customers in a positive, enthusiastic, professional manner while educating them on the large variety of carwash and detail shop services that we offer. These professional services have been expertly crafted to greatly improve our customer's vehicle appearance and enjoyment. This is a fast-paced position that requires the ability to multi-task with a sense of urgency while meeting customer service expectations. Additional responsibilities include, but are not limited to, transacting customer purchases, maintaining proper cash handling procedures, and following all position-specific procedures. Customer Advisors will begin their training in the position of Delta Tech to enhance their product knowledge. Individuals in this position can enjoy working in the carwash and/or detail shop, depending on the location's staffing needs. The pay rate for this position is $15.75 per hour plus commission.
Requirements
Excellent communication skills and a passion for providing great customer service
A competitive drive with a desire to rank amongst the top advisors in the company
Unwavering honesty and integrity
Courteous and professional demeanor
Prior experience in automotive service writing, customer service, a

In [0]:
# DO NOT RUN - took 17m to run

average_sentiment_score = job_summary_skills_spk.agg(avg(col("sentiment_score"))
                                                     .alias("average_sentiment_score"))
average_sentiment_score.show()

# average_sentiment_score: 0.9744135490001786

+-----------------------+
|average_sentiment_score|
+-----------------------+
|     0.9744135490001786|
+-----------------------+



In [0]:
# DO NOT RUN - took 50m to run

# calculate the minimum and maximum values of the 'sentiment_score' column
min_value = job_summary_skills_spk.selectExpr("min(sentiment_score)").collect()[0][0]
max_value = job_summary_skills_spk.selectExpr("max(sentiment_score)").collect()[0][0]

# as said before, we sampled 100,000 rows, so the total count of 
# 'job_summary_skills_spk' dataframe is 100,000
total_count = 100000

# calculate the percentage of values ranging from 0.9 to 1
range_count = job_summary_skills_spk.filter((col('sentiment_score') >= 0.9) & 
                                            (col('sentiment_score') <= 1)).count()
percentage_range = (range_count / total_count) * 100

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")
print(f"Percentage of values ranging from 0.9 to 1: {percentage_range:.2f}%")

Minimum value: -0.0
Maximum value: 0.9999
Percentage of values ranging from 0.9 to 1: 96.14%


*The VADER sentiment intensity uses a dictionary of words and rules to determine the sentiment of a piece of text. It uses a valence score for each word to determine its positivity or negativity. VADER also takes into account the intensity of the sentiment, which can be determined by capitalization and punctuation.* 

*The sentiment score ranges from -4 to +4, where scores close to 4 indicate strong positive sentiment, scores close to -4 indicate strong negative sentiment, and scores close to zero indicate neutral sentiment.* 

*The average sentiment score is 0.974, and the percentage of values ranging from 0.9 to 1 is 96.14%, so this approach doesn't really help us distinguish between different job summaries, in terms of sentiment.*

In [0]:
# SECOND ATTEMPT: Vivekn sentiment model

from pyspark.ml import Pipeline, PipelineModel
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *

document = DocumentAssembler() \
.setInputCol("job_summary") \
.setOutputCol("document")

token = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("token")

normalizer = Normalizer() \
.setInputCols(["token"]) \
.setOutputCol("normal")

vivekn =  ViveknSentimentModel.pretrained() \
.setInputCols(["document", "normal"]) \
.setOutputCol("result_sentiment")

finisher = Finisher() \
.setInputCols(["result_sentiment"]) \
.setOutputCols("sentiment_indicator")

pipeline = Pipeline().setStages([document, token, normalizer, vivekn, finisher])

# fit to data
pipelineModel = pipeline.fit(job_summary_skills_spk)
job_summary_skills_spk = pipelineModel.transform(job_summary_skills_spk)

sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[OK!]


In [0]:
# SECOND ATTEMPT RESULTS
display(job_summary_skills_spk.select('job_summary', 'sentiment_indicator', 'summary_length').limit(10))

job_summary sentiment_indicator summary_length Want to learn more about this role and Jobot? Click our Jobot logo and follow our LinkedIn page!
Job details
Lead Superintendent Role with a National well regarded Real Estate Developer, all LOCAL Multi-Family Projects, Great Benefits, Bonus and Truck Allowance!
This Jobot Job is hosted by Ben Kimbrel
Are you a fit? Easy Apply now by clicking the "Easy Apply" button and sending us your resume.
Salary $120,000 - $150,000 per year
A Bit About Us
We are a National Multi-Family Real Estate Developer with a large presence in Florida. Our core values are centered around our people and our clients. We believe that putting your employees first is not only the right thing to do, but it also gives us the competitive edge to ensure our clients have an excellent experience! Over the years our reputation for providing our people with the very best has caught the eye of some reputable companies who have chosen to use us as their Prime Contractor! We build a variety of Multi-Family projects including wood-frame over podium and concrete multi story high-rises.
If you are a Construction Superintendent with 5+ years of multi-family experience then please apply!
Why join us?
Do you want to work on great projects and be part of a top Firm!?
Competitive Base Salary!
Competitive Bonus Package!
Extremely Competitive Benefits Package!
Flexible Work Schedules!
Company Truck/Truck Allowance!
Local Work to the Fort Myers Area!
Job Details
Is your background a fit? Apply if you meet this criteria
5+ years’ experience in Construction field supervision
Experience w/ multi-family ground up construction
Experience with projects $20M+
We can offer you the opportunity to work on great Projects and make a meaningful impact on today's society and the next generation! Help us revolutionize the construction industry and enjoy accelerated career growth with a great company!
Interested in hearing more? Easy Apply now by clicking the "Easy Apply" button.
Want to learn more about this role and Jobot?
Click our Jobot logo and follow our LinkedIn page!
Show more
Show less List(negative) 312 Job Details
Description
Join Our Award Winning Team!
With flexible schedules, team member discounts, scholarship opportunities, and fun recognition programs, Delta Sonic offers a variety of career paths for you to grow with us. Recognized nationally as a Top Place to Work, we take great pride in providing outstanding opportunities and a positive work culture.
Team Members Enjoy
Career opportunities
Flexible schedules
Team member discounts
Scholarship opportunities
Benefit options
A fun, team-oriented culture
Equal opportunity employer
Responsibilities
As a front-line representative of the Delta Sonic Team, Customer Advisors are responsible for greeting all customers in a positive, enthusiastic, professional manner while educating them on the large variety of carwash and detail shop services that we offer. These professional services have been expertly crafted to greatly improve our customer's vehicle appearance and enjoyment. This is a fast-paced position that requires the ability to multi-task with a sense of urgency while meeting customer service expectations. Additional responsibilities include, but are not limited to, transacting customer purchases, maintaining proper cash handling procedures, and following all position-specific procedures. Customer Advisors will begin their training in the position of Delta Tech to enhance their product knowledge. Individuals in this position can enjoy working in the carwash and/or detail shop, depending on the location's staffing needs. The pay rate for this position is $15.75 per hour plus commission.
Requirements
Excellent communication skills and a passion for providing great customer service
A competitive drive with a desire to rank amongst the top advisors in the company
Unwavering honesty and integrity
Courteous and professional demeanor
Prior experience in automotive service writing, custome

In [0]:
# DO NOT RUN - took 2h to run

# calculate the percentage of positive and negative job summaries

# as said before, we sampled 100,000 rows, so the total count of 
# 'job_summary_skills_spk' dataframe is 100,000
total_count = 100000

exploded_indicators = job_summary_skills_spk.withColumn("indicator_value", 
                                                        explode(col("sentiment_indicator")))

positive_count = exploded_indicators.filter(col('indicator_value') == 'positive').count()
negative_count = exploded_indicators.filter(col('indicator_value') == 'negative').count()

positive_percentage = (positive_count / total_count) * 100
negative_percentage = (negative_count / total_count) * 100

print(f"Positive Percentage: {positive_percentage:.2f}%")
print(f"Negative Percentage: {negative_percentage:.2f}%")

# Positive Percentage: 57.06%
# Negative Percentage: 42.94%

Positive Percentage: 57.06%
Negative Percentage: 42.94%


In [0]:
# convert the positive\negative values to binary values

# define a UDF to map the array values to integers
def map_indicator(arr):
    if "positive" in arr:
        return 1
    else:
        return 0

map_indicator_udf = udf(map_indicator, returnType=IntegerType())

job_summary_skills_spk = job_summary_skills_spk.withColumn('numeric_sentiment', 
                                                           map_indicator_udf('sentiment_indicator'))

In [0]:
display(job_summary_skills_spk.limit(100))

job_link job_summary job_skills summary_length summary_newline_count sentiment_score sentiment_indicator readability_score numeric_sentiment https://www.linkedin.com/jobs/view/construction-superintendent-multi-family-projects-at-jobot-3804045404 Want to learn more about this role and Jobot? Click our Jobot logo and follow our LinkedIn page!
Job details
Lead Superintendent Role with a National well regarded Real Estate Developer, all LOCAL Multi-Family Projects, Great Benefits, Bonus and Truck Allowance!
This Jobot Job is hosted by Ben Kimbrel
Are you a fit? Easy Apply now by clicking the "Easy Apply" button and sending us your resume.
Salary $120,000 - $150,000 per year
A Bit About Us
We are a National Multi-Family Real Estate Developer with a large presence in Florida. Our core values are centered around our people and our clients. We believe that putting your employees first is not only the right thing to do, but it also gives us the competitive edge to ensure our clients have an excellent experience! Over the years our reputation for providing our people with the very best has caught the eye of some reputable companies who have chosen to use us as their Prime Contractor! We build a variety of Multi-Family projects including wood-frame over podium and concrete multi story high-rises.
If you are a Construction Superintendent with 5+ years of multi-family experience then please apply!
Why join us?
Do you want to work on great projects and be part of a top Firm!?
Competitive Base Salary!
Competitive Bonus Package!
Extremely Competitive Benefits Package!
Flexible Work Schedules!
Company Truck/Truck Allowance!
Local Work to the Fort Myers Area!
Job Details
Is your background a fit? Apply if you meet this criteria
5+ years’ experience in Construction field supervision
Experience w/ multi-family ground up construction
Experience with projects $20M+
We can offer you the opportunity to work on great Projects and make a meaningful impact on today's society and the next generation! Help us revolutionize the construction industry and enjoy accelerated career growth with a great company!
Interested in hearing more? Easy Apply now by clicking the "Easy Apply" button.
Want to learn more about this role and Jobot?
Click our Jobot logo and follow our LinkedIn page!
Show more
Show less Construction supervision, Multifamily groundup construction, Projects over $20M, Flexible work schedules, Company truck/truck allowance, Local work in the Fort Myers area 312 27 0.9979 List(negative) 58.79 0 https://www.linkedin.com/jobs/view/customer-advisor-1-at-delta-sonic-3805270802 Job Details
Description
Join Our Award Winning Team!
With flexible schedules, team member discounts, scholarship opportunities, and fun recognition programs, Delta Sonic offers a variety of career paths for you to grow with us. Recognized nationally as a Top Place to Work, we take great pride in providing outstanding opportunities and a positive work culture.
Team Members Enjoy
Career opportunities
Flexible schedules
Team member discounts
Scholarship opportunities
Benefit options
A fun, team-oriented culture
Equal opportunity employer
Responsibilities
As a front-line representative of the Delta Sonic Team, Customer Advisors are responsible for greeting all customers in a positive, enthusiastic, professional manner while educating them on the large variety of carwash and detail shop services that we offer. These professional services have been expertly crafted to greatly improve our customer's vehicle appearance and enjoyment. This is a fast-paced position that requires the ability to multi-task with a sense of urgency while meeting customer service expectations. Additional responsibilities include, but are not limited to, transacting customer purchases, maintaining proper cash handling procedures, and following all position-specific procedures. Customer Advisors will begin their training in the position of Delta Tech to enhance their product knowledge. Individuals in this positi

*The Vivekn sentiment model uses Naive-Bayes classifier for sentiment analysis. For each job summary, it gives a value: positive or negative. The Vivekn approach has shown promising results in various studies and has been used in several applications, including sentiment analysis of social media data, movie reviews, and product reviews.*

*The positive percentage of positive summaries is 57.06%, and the percentage of negative summaries is 42.94%.*

*This approach is **more informative** than the first approach, because it gives us a way to distinguish the difference between summaries.*

##### Measure 4: english readability

In [0]:
pip install textstat

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# We used the Flesch Reading Ease formula

import textstat
from pyspark.sql.types import DoubleType

# Define a UDF to calculate Flesch Reading Ease score
def calculate_flesch_reading_ease(text):
    try:
        return textstat.flesch_reading_ease(text)
    except:
        return None

# Register the UDF
flesch_reading_ease_udf = udf(calculate_flesch_reading_ease, DoubleType())

# Apply the UDF to calculate Flesch Reading Ease score
job_summary_skills_spk = job_summary_skills_spk.withColumn
("readability_score", flesch_reading_ease_udf(job_summary_skills_spk["job_summary"]))

In [0]:
# DO NOT RUN 

average_flesch_score = job_summary_skills_spk.agg(avg(col("readability_score")).alias("average_score"))
average_flesch_score.show()

# average_flesch_score: 27.240449799999972

+------------------+
|     average_score|
+------------------+
|27.240449799999972|
+------------------+



*The Flesch Reading Ease formula gives a readability score for a given english text. The maximum score is 121.22, there is no limit on how low the score can be (a negative score is valid).*

*Scores ranging from 30-50 indicates that the english text is difficult to read, and requires a college level to fully understand it. Scores ranging from 10-30 indicates that the english text is very difficult to read, and requires a college graduate level to fully understand it. Scores ranging from 0-10 indicates that the english text is extremely difficult to read, and requires a professional level to fully understand it.*

*The average readability score is 27.24, which is suitable for college graduates' level. This score fairly reflects reality, because most companies search for graduates or students who are about to graduate.*

##### Measure 5: key word density

In [0]:
def remove_missing_from_cols(df, col_list):
    for column in col_list:
        df = df.filter(col(column).isNotNull())
        df = df.filter(col(column) != "N/A")
        df = df.filter(col(column) != "N\A")
        df = df.filter(col(column) != "None")
        df = df.filter(col(column) != '')
    return df

all_profiles = profiles.select("*", col("experience.description").alias("experience_description"))

all_profiles_exploded = all_profiles.withColumn
("experience_description", explode(col("experience_description")))
all_profiles_exploded = all_profiles_exploded.dropDuplicates(['experience_description'])
all_profiles_exploded = remove_missing_from_cols(all_profiles_exploded, ['experience_description'])

profiles_companies = all_profiles_exploded.join
(companies, all_profiles_exploded['current_company:company_id'] == companies['id'], "inner")

# count for 'profiles_companies': 92296

In [0]:
# from pyspark.ml.feature import Tokenizer

stopwords = StopWordsCleaner().getStopWords()

document = DocumentAssembler() \
    .setInputCol("experience_description") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

token = Tokenizer() \
    .setInputCols("sentence") \
    .setOutputCol("token") \
    .setContextChars(["(", ")", "?", "!", ".", ","])

keywords = YakeKeywordExtraction() \
    .setInputCols("token") \
    .setOutputCol("keywords") \
    .setMinNGrams(1) \
    .setMaxNGrams(1)\
    .setNKeywords(10)\
    .setStopWords(stopwords)

yake_pipeline = Pipeline(stages=[document, sentenceDetector, token, keywords])

In [0]:
result = yake_pipeline.fit(profiles_companies).transform(profiles_companies)
result = result.withColumn('unique_keywords', F.array_distinct("keywords.result"))

In [0]:
# DO NOT RUN - it spoils the regular display

from IPython.core.display import display, HTML

def highlight(text, keywords):
    for k in keywords:
        text = (re.sub(r'(\b%s\b)'%k, r'<span style="background-color: yellow;">\1</span>', 
                       text, flags=re.IGNORECASE))
    return text

highlight_udf = udf(highlight, StringType())
result = result.withColumn("highlighted_keywords",highlight_udf('experience_description','unique_keywords'))

for r in result.select("highlighted_keywords").limit(20).collect():
    display(HTML(r.highlighted_keywords))
    print("\n\n")

Volunteer graphic designer on projects to help promote the non-profit solar energy industry . Volunteer graphic designer on projects to help promote the non-profit solar energy industry .

TP Printing covers Clark, Western Marathon and Taylor Counties with publications such as: The Tribune -Phonograph, The Record -Review, Central Wisconsin Shopper and Tribune Record Gleaner . I was responsible for creating visual solutions for clients using a mix of creative skills and commercial awareness while having a good attitude and an open mind for constructive criticism. Ad design; photography editing; newspaper page layout; worked with tight deadlines; designed signage, banners, posters, business materials for print; collaborated with sales, editorial staff and customers. Show less

Administrative support and managed daily operations of the facility, scheduled building use by the Center , its tenants and occasional users, increased social media marketing , managed WordPress website , designed marketing materials and seasonal brochures, managed membership database , created monthly e-newsletters, analyzed social media and website traffic. - Single-handedly converted membership database of 10,000+ from Microsoft Access to Salsa Labs Donor Management System. - Position started as Administrative Assistant and evolved to Office Manager and Rental Coordinator. Show less

The Star News covers Taylor County with publications such as: The Star News , The Star News Shopper . I create visual solutions for clients using a mix of creative skills and commercial awareness while having a good attitude and an open mind for constructive criticism. The Star News covers Taylor County with publications such as: The Star News , The Star News Shopper . I create visual solutions for clients using a mix of creative skills and commercial awareness while having a good attitude and an open mind for constructive criticism.

Krystal Holographics is a wholly owned subsidiary of Krystal Tech . International Krystal Holographics is a wholly owned subsidiary of Krystal Tech . International

...

After being diagnosed with cancer and kicking it's trash, I decided I wasn't going to waste another moment being weak and tired all the time. I started running and completed my first 5k, followed by a 10k, relay races, half marathon, Tough Mudder , and most recently the Spartan Beast . I was introduced to indoor cycling through Kinetic's boot camp and now prefer cycling to running any day. As long as the beat is good, you can ride to any song, and as long as you are willing to put in the work, I can help you get the workout you need. Come take a ride with me. Show less

I was a bar-back for Brown Truck Brewery the summer after I graduated high school . I was a bar-back for Brown Truck Brewery the summer after I graduated high school .

I shadowed the Athletic Trainer at my high school , Westchester Country Day School . I shadowed during our Varsity Boy's Soccer season and some of the Varsity Boy's basketball season . I learned some exercises for rehab, signs of possible injuries, and how to bag ice for post game cool down. I also helped with refilling water and any last minute setting up. I shadowed the Athletic Trainer at my high school , Westchester Country Day School . I shadowed during our Varsity Boy's Soccer season and some of the Varsity Boy's basketball season . I learned some exercises for rehab, signs of possible injuries, and how to bag ice for post game cool down. I also helped with refilling water and any last minute setting up.

I began my own at-home baking business - The Tasty Turtle . I began my own at-home baking business - The Tasty Turtle .

Within my duration at Dublin Retirement Village , I held many different roles that assisted the residents in various ways. I followed the appropriate safety procedures to prepare and present food while also responding to complaints swiftly and professionally. I was able to continue working effectively in a high stress environment to ensure flawless service. Moreover, I would assist other departments within the building such as the front desk by delivering packages and helping residents move their belongings in and out. In the midst of the Covid-19 pandemic, I helped promote and enact safety precautions to keep the residents safe. I would take and record temperatures twice daily and practice the proper donning and removal of necessary PPE . Show less

In [0]:
result_last_k_columns = result.select('industries', 'experience_description', 'document', 'sentence', 'token', 'keywords', 'unique_keywords')

In [0]:
result_last_k_columns1 = result_last_k_columns.select(
    col("industries"),
    col("unique_keywords")
)
# Explode the unique_keywords array to create a row for each keyword
df_exploded = result_last_k_columns1.select(
    col("industries"),
    explode(col("unique_keywords")).alias("keyword")
)

In [0]:
# Count the occurrences of each keyword within each industry
df_keyword_counts = df_exploded.groupBy("industries", "keyword").agg(
    count("*").alias("count"))

In [0]:
# Define a window partitioned by industries and ordered by count in descending order
windowSpec = Window.partitionBy("industries").orderBy(col("count").desc())

# Apply row_number over the windowSpec to each row
df_ranked_keywords = df_keyword_counts.withColumn("row_number", row_number().over(windowSpec))

# Filter to keep only the top k rows for each industry
k = 50 
df_top_k_keywords = df_ranked_keywords.filter(col("row_number") <= k)

In [0]:
# DO NOT RUN - took 1h and 40m to run
display(df_top_k_keywords)

industries,keyword,count,row_number
Accounting,tax,14,1
Accounting,clients,14,2
Accounting,accounting,14,3
Accounting,client,10,4
Accounting,sales,9,5
Accounting,maintained,9,6
Accounting,financial,9,7
Accounting,responsible,8,8
Accounting,process,7,9
Accounting,state,7,10


In [0]:
meta_industries_12 = {
    'Furniture and Home Furnishings Manufacturing': 'Manufacturing',
    'Investment Banking': 'Financial and Investment',
    'Architecture and Planning': 'Services',
    'Wholesale': 'Services',
    'Travel Arrangements': 'Services',
    'Ranching': 'Miscellaneous',
    'Hospitals and Health Care': 'Healthcare and Medical',
    'Book and Periodical Publishing': 'Services',
    'Printing Services': 'Services',
    'Professional Training and Coaching': 'Services',
    'Computers and Electronics Manufacturing': 'Manufacturing',
    'Shipbuilding': 'Manufacturing',
    'Public Policy Offices': 'Government and Public Policy',
    'Software Development': 'Technology',
    'Outsourcing and Offshoring Consulting': 'Services',
    'Retail Groceries': 'Retail and Consumer Goods',
    'Education Administration Programs': 'Education and Training',
    'Plastics Manufacturing': 'Manufacturing',
    'Renewable Energy Semiconductor Manufacturing': 'Manufacturing',
    'Computer Networking Products': 'Technology',
    'Events Services': 'Services',
    'Information Services': 'Services',
    'Food and Beverage Services': 'Services',
    'Semiconductor Manufacturing': 'Manufacturing',
    'Business Consulting and Services': 'Services',
    'Insurance': 'Services',
    'Financial Services': 'Services',
    'Wireless Services': 'Services',
    'Computer Hardware Manufacturing': 'Technology',
    'Public Safety': 'Services',
    'Maritime Transportation': 'Transportation and Logistics',
    'Tobacco Manufacturing': 'Manufacturing',
    'Writing and Editing': 'Services',
    'Veterinary Services': 'Services',
    'Staffing and Recruiting': 'Services',
    'Accounting': 'Services',
    'International Affairs': 'Government and Public Policy',
    'Spectator Sports': 'Miscellaneous',
    'Glass, Ceramics and Concrete Manufacturing': 'Manufacturing',
    'Chemical Manufacturing': 'Manufacturing',
    'Mining': 'Miscellaneous',
    'E-Learning Providers': 'Technology',
    'Security and Investigations': 'Services',
    'Translation and Localization': 'Services',
    'Automation Machinery Manufacturing': 'Technology',
    'Computer and Network Security': 'Technology',
    'Political Organizations': 'Government and Public Policy',
    'Environmental Services': 'Government and Public Policy',
    'Oil and Gas': 'Miscellaneous',
    'Real Estate': 'Real Estate and Construction',
    'Think Tanks': 'Government and Public Policy',
    'Executive Offices': 'Miscellaneous',
    'Law Practice': 'Services',
    'Nanotechnology Research': 'Miscellaneous',
    'International Trade and Development': 'Government and Public Policy',
    'Personal Care Product Manufacturing': 'Manufacturing',
    'Philanthropic Fundraising Services': 'Services',
    'Entertainment Providers': 'Media and Entertainment',
    'Market Research': 'Media and Entertainment',
    'Movies, Videos, and Sound': 'Media and Entertainment',
    'Sporting Goods Manufacturing': 'Manufacturing',
    'Graphic Design': 'Services',
    'Technology, Information and Internet': 'Technology',
    'IT Services and IT Consulting': 'Technology',
    'Retail Office Equipment': 'Retail and Consumer Goods',
    'Wholesale Import and Export': 'Services',
    'Capital Markets': 'Financial and Investment',
    'Law Enforcement': 'Services',
    'Freight and Package Transportation': 'Transportation and Logistics',
    'Industrial Machinery Manufacturing': 'Manufacturing',
    'Non-profit Organizations': 'Miscellaneous',
    'Retail Art Supplies': 'Retail and Consumer Goods',
    'Animation and Post-production': 'Media and Entertainment',
    'Transportation, Logistics, Supply Chain and Storage': 'Transportation and Logistics',
    'Aviation and Aerospace Component Manufacturing': 'Transportation and Logistics',
    'Fundraising': 'Financial and Investment',
    'Railroad Equipment Manufacturing': 'Transportation and Logistics',
    'Construction': 'Real Estate and Construction',
    'Investment Management': 'Financial and Investment',
    'Utilities': 'Miscellaneous',
    'Retail Luxury Goods and Jewelry': 'Retail and Consumer Goods',
    'Warehousing and Storage': 'Transportation and Logistics',
    'Media Production': 'Media and Entertainment',
    'Gambling Facilities and Casinos': 'Media and Entertainment',
    'Defense and Space Manufacturing': 'Manufacturing',
    'Facilities Services': 'Services',
    'Government Relations Services': 'Government and Public Policy',
    'Advertising Services': 'Media and Entertainment',
    'Paper and Forest Product Manufacturing': 'Manufacturing',
    'Packaging and Containers Manufacturing': 'Manufacturing',
    'Telecommunications': 'Technology',
    'Medical Equipment Manufacturing': 'Healthcare and Medical',
    'Beverage Manufacturing': 'Manufacturing',
    'Restaurants': 'Retail and Consumer Goods',
    'Leasing Non-residential Real Estate': 'Real Estate and Construction',
    'Newspaper Publishing': 'Media and Entertainment',
    'Armed Forces': 'Miscellaneous',
    'Appliances, Electrical, and Electronics Manufacturing': 'Manufacturing',
    'Hospitality': 'Services',
    'Pharmaceutical Manufacturing': 'Healthcare and Medical',
    'Research Services': 'Services',
    'Retail Apparel and Fashion': 'Retail and Consumer Goods',
    'Photography': 'Media and Entertainment',
    'Wellness and Fitness Services': 'Services',
    'Truck Transportation': 'Transportation and Logistics',
    'Consumer Services': 'Services',
    'Wholesale Building Materials': 'Services',
    'Human Resources Services': 'Services',
    'Airlines and Aviation': 'Transportation and Logistics',
    'Machinery Manufacturing': 'Manufacturing',
    'Individual and Family Services': 'Services',
    'Motor Vehicle Manufacturing': 'Manufacturing',
    'Performing Arts': 'Media and Entertainment',
    'Museums, Historical Sites, and Zoos': 'Media and Entertainment',
    'Broadcast Media Production and Distribution': 'Media and Entertainment',
    'Banking': 'Financial and Investment',
    'Recreational Facilities': 'Miscellaneous',
    'Government Administration': 'Government and Public Policy',
    'Public Relations and Communications Services': 'Media and Entertainment',
    'Fisheries': 'Miscellaneous',
    'Medical Practices': 'Healthcare and Medical',
    'Religious Institutions': 'Miscellaneous',
    'Online Audio and Video Media': 'Media and Entertainment',
    'Artists and Writers': 'Miscellaneous',
    'Biotechnology Research': 'Healthcare and Medical',
    'Legal Services': 'Services',
    'Retail': 'Retail and Consumer Goods',
    'Civil Engineering': 'Services',
    'Libraries': 'Miscellaneous',
    'Alternative Dispute Resolution': 'Miscellaneous',
    'Manufacturing': 'Miscellaneous',
    'Design Services': 'Services',
    'Dairy Product Manufacturing': 'Manufacturing',
    'Higher Education': 'Education and Training',
    'Civic and Social Organizations': 'Miscellaneous',
    'Textile Manufacturing': 'Manufacturing',
    'Venture Capital and Private Equity Principals': 'Financial and Investment',
    'Mental Health Care': 'Healthcare and Medical',
    'Musicians': 'Media and Entertainment',
    'Farming': 'Miscellaneous',
    'Computer Games': 'Media and Entertainment',
    'Strategic Management Services': 'Services',
    'Food and Beverage Manufacturing': 'Manufacturing',
    'Primary and Secondary Education': 'Education and Training',
    'Alternative Medicine': 'Healthcare and Medical',
    'Legislative Offices': 'Services',
    'Administration of Justice': 'Services',
    'Mobile Gaming Apps': 'Media and Entertainment'
}

In [0]:
from pyspark.sql import Row

meta_industry_df = spark.createDataFrame([Row(industries=k, meta_industry=v) 
                                          for k, v in meta_industries_12.items()])
df_with_meta = df_top_k_keywords.join(meta_industry_df, on='industries', how='left')
df_with_meta = df_with_meta.fillna({'meta_industry': 'DefaultMetaIndustry'})

In [0]:
@udf
def map_meta_industry(industries):
    if industries in meta_industries_12:
        return meta_industries_12[industries]
    else:
        return None
df_with_meta = df_top_k_keywords.withColumn('meta_industry', map_meta_industry(col('industries')))


In [0]:
display(df_with_meta)

#### Analyzing to reach an attractivity formula:

*check if there's a correlation between 'summary_length' and 'summary_newline_count'*

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from scipy.stats import pearsonr

# Assemble the features into a single vector
assembler = VectorAssembler(inputCols=["summary_length", "summary_newline_count"], outputCol="features")
summary_skills_assembled = assembler.transform(job_summary_skills_spk)

# Calculate correlation
pearson_corr = Correlation.corr(summary_skills_assembled, "features").head()
print("Pearson correlation matrix:\n", pearson_corr[0])

# Extract the correlation value between summary_length and summary_newline_count
corr_value = pearson_corr[0].toArray()[0, 1]

# Calculate the p-value
p_value = pearsonr(job_summary_skills_spk.select("summary_length").rdd.flatMap(lambda x: x).collect(),
                   job_summary_skills_spk.select("summary_newline_count").rdd.flatMap(lambda x: x).collect())[1]

print("Correlation between summary_length and summary_newline_count:", corr_value)
print("P-value:", p_value)

Pearson correlation matrix:
 DenseMatrix([[1.        , 0.73957244],
             [0.73957244, 1.        ]])
Correlation between summary_length and summary_newline_count: 0.7395724385646975
P-value: 0.0


*The pearson correlation is 0.73 ---> there is a high correlation between the two columns, also the p-value is very small which means that's it's statisticlly significant.*